The implementation of the program is entirely mine, except for the Neural Network which was originally inspired and implemented by Eu Jin Lok's work on Audio Processing https://www.kaggle.com/ejlok1/

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

import os
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import json
import pickle
import time
from google.colab import drive

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob

import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans, FeatureAgglomeration, MeanShift, AgglomerativeClustering
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
import os

In [ ]:
#print(dirList)
M_train = np.loadtxt('digit-recognizer/train.csv', delimiter=',', skiprows=1)
MNISTDF_train = pd.read_csv("digit-recognizer/train.csv")
#MNISTDF_test = pd.read_csv("digit-recognizer/test.csv")
#MNISTDF_sample = pd.read_csv("digit-recognizer/sample_submission.csv")
display(M_train)
#display(MNISTDF_test)
#display(MNISTDF_sample)

In [ ]:
#MNISTDF_test = pd.concat([MNISTDF_test, MNISTDF_sample], axis = 1)
#MNISTDF_test = MNISTDF_test.drop(columns=["ImageId"])
#print(MNISTDF_test)

In [ ]:
MNISTDF_train_binary_one = MNISTDF_train[MNISTDF_train.label == 1]
MNISTDF_train_binary_zero = MNISTDF_train[MNISTDF_train.label == 0]
#MNISTDF_test_binary_one = MNISTDF_test[MNISTDF_test.Label == 1]
#MNISTDF_test_binary_zero = MNISTDF_test[MNISTDF_test.Label == 0]
#display(MNISTDF_test_binary_one)
#display(MNISTDF_test_binary_zero)

In [ ]:
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero], ignore_index=True)
#MINST_test = pd.concat([MNISTDF_test_binary_one, MNISTDF_test_binary_zero], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
#MINST_test = MINST_test.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
#display(MINST_test)

In [ ]:
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)

In [ ]:
sampleindex = np.random.randint(0,1000)
sample = trainingset[0, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[0])

In [ ]:
sampleindex = np.random.randint(0,1000)
sample = trainingset[4, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[4])

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x_training, y_training, test_size = 0.1, shuffle = True, random_state = 42)
x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs').fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")


In [ ]:
svmClass = SVC(kernel = 'linear')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.cluster import KMeans, FeatureAgglomeration, MeanShift, AgglomerativeClustering
# from sklearn.ensemble import AdaBoostClassifier

DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
KMClassList = []
for i in range(1, 10):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)



In [ ]:
KMClassList = []
for i in range(1, 10):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

MeanClass = MeanShift()
MeanClass.fit(x_train, y_train)
predictedMeanClass = MeanClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

AlgoClass = AgglomerativeClustering(n_clusters=2, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")


In [ ]:
from sklearn.cluster import AgglomerativeClustering
AlgoClass = AgglomerativeClustering(n_clusters=2, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")


In [ ]:
#print(np.unique(vFeaturedf["VoxCeleb1 ID"].shape))
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)

In [ ]:
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In the previous example, the accuracy score was 0%. This means for the previous example every prediction was the opposite.

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(2)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=15, validation_data=(x_test, y_test))

In [ ]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
# # Actual labels
# actual = y_test.argmax(axis=1)
# actual = actual.astype(int).flatten()
# actual = (lb.inverse_transform((actual)))
# actual = pd.DataFrame({'actualvalues': actual})

# # Lets combined both of them into a single dataframe
# finaldf = actual.join(preds)
# finaldf[:]

# print(y_test.argmax(axis=1))
# finalY_test = y_test.argmax(axis=1)
# predictedClassLabels = []
# actualClassLabels = []
# for index, i in enumerate(preds['predictedvalues']):
#     predictedClassLabels.append(TransformedClassLabels[i])
#     actualClassLabels.append(TransformedClassLabels[i])

# #print(predictedClassLabels)
# #print(actualClassLabels)
# data = [predictedClassLabels, actualClassLabels]
# print(data[0])
# print(data[1])
# finaldf = pd.DataFrame(data, columns=['Predicted Class Labels', "Actual Class Labels"])

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_two = MNISTDF_train[MNISTDF_train.label == 2]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[12990, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[12990])

In [ ]:
print(y_test)
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
print(np.unique(y_test))
svmClass = SVC(kernel = 'linear')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
svmClass = SVC(kernel = 'rbf')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

In [ ]:
print(np.unique(y_test))
MeanClass = MeanShift()
MeanClass.fit(x_train, y_train)
predictedMeanClass = MeanClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=3, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
print(np.unique(y_test))
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(3)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
print(np.unique(y_test))
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_three = MNISTDF_train[MNISTDF_train.label == 3]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two, MNISTDF_train_three], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[1, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[1])

In [ ]:
print(np.unique(y_test))
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
svmClass = SVC(kernel = 'linear')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")
print()

In [ ]:
print()
svmClass = SVC(kernel = 'rbf')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

In [ ]:
# print(np.unique(y_test))
# MeanClass = MeanShift()
# MeanClass.fit(x_train, y_train)
# predictedMeanClass = MeanClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=4, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
#print(np.unique(vFeaturedf["VoxCeleb1 ID"].shape))
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(4)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_four = MNISTDF_train[MNISTDF_train.label == 4]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two, MNISTDF_train_three, MNISTDF_train_four], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[1, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[1])

In [ ]:
print(np.unique(y_test))
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial").fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
svmClass = SVC(kernel = 'linear')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")
print()

In [ ]:
print()
svmClass = SVC(kernel = 'rbf')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMCLassList)

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=5, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
# print(KMClassList)
# print(np.unique(y_test))
# MeanClass = MeanShift()
# MeanClass.fit(x_train, y_train)
# predictedMeanClass = MeanClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

In [ ]:
#print(np.unique(vFeaturedf["VoxCeleb1 ID"].shape))
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(5)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_five = MNISTDF_train[MNISTDF_train.label == 5]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two, MNISTDF_train_three, MNISTDF_train_four, MNISTDF_train_five], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[25209, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[25209])

In [ ]:
print(np.unique(y_test))
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial").fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
svmClass = SVC(kernel = 'linear')
svmClass.fit(x_train, y_train)
predictedSVM = svmClass.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
#print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")
print()

In [ ]:
# print()
# svmClass = SVC(kernel = 'rbf')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# #print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=6, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
# print(np.unique(y_test))
# MeanClass = MeanShift()
# MeanClass.fit(x_train, y_train)
# predictedMeanClass = MeanClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")


In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
print(np.unique(y_test))
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_six = MNISTDF_train[MNISTDF_train.label == 6]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two, MNISTDF_train_three, MNISTDF_train_four, MNISTDF_train_five, MNISTDF_train_six], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[29344, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[29344])

In [ ]:
print(np.unique(y_test))
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial").fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
# svmClass = SVC(kernel = 'linear')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# #print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")
print()

# print()
# svmClass = SVC(kernel = 'rbf')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# #print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

# print(KMClassList)
# print(np.unique(y_test))
# MeanClass = MeanShift()
# MeanClass.fit(x_train, y_train)
# predictedMeanClass = MeanClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=7, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)



In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(7)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
print(np.unique(y_test))
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_seven = MNISTDF_train[MNISTDF_train.label == 7]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two, MNISTDF_train_three, MNISTDF_train_four,
                         MNISTDF_train_five, MNISTDF_train_six, MNISTDF_train_seven], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[78, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[2])

In [ ]:
print(np.unique(y_test))
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial").fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
# svmClass = SVC(kernel = 'linear')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")
print()

# print()
# svmClass = SVC(kernel = 'rbf')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# #print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

# print(KMClassList)
# print(np.unique(y_test))
# MeanClass = MeanShift()
# MeanClass.fit(x_train, y_train)
# predictedMeanClass = MeanClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=8, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(8)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
print(np.unique(y_test))
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_eight = MNISTDF_train[MNISTDF_train.label == 8]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two, MNISTDF_train_three, MNISTDF_train_four,
                         MNISTDF_train_five, MNISTDF_train_six, MNISTDF_train_seven, MNISTDF_train_eight], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[37808, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[37808])

In [ ]:
print(np.unique(y_test))
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial").fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
# svmClass = SVC(kernel = 'linear')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")
print()

# print()
# svmClass = SVC(kernel = 'rbf')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# #print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

# print(KMClassList)
# print(np.unique(y_test))
# MeanClass = MeanShift()
# MeanClass.fit(x_train, y_train)
# predictedMeanClass = MeanClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=9, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(9)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()


In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
print(np.unique(y_test))
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------

In [ ]:
MNISTDF_train_nine = MNISTDF_train[MNISTDF_train.label == 9]
MINST_train = pd.concat([MNISTDF_train_binary_one, MNISTDF_train_binary_zero, MNISTDF_train_two, MNISTDF_train_three, MNISTDF_train_four,
                         MNISTDF_train_five, MNISTDF_train_six, MNISTDF_train_seven, MNISTDF_train_eight, MNISTDF_train_nine], ignore_index=True)
MINST_train = MINST_train.sample(frac=1).reset_index(drop = True)
display(MINST_train)
exportingMINST_train = MINST_train.drop(columns=['label'])
exportCheck = exportingMINST_train.to_csv("convert_to_numpy.csv", index=False)
n_inputs = 784
x_training = MINST_train.iloc[:, MINST_train.columns != 'label']
y_training = MINST_train.iloc[:, MINST_train.columns == 'label'] # Extract the last 7 elements - a one hot category encoding
sampleImage = x_training.iloc[0].to_numpy

x_train, x_test, y_train, y_test = train_test_split(MINST_train.drop(['label'], axis = 1),
                                                    MINST_train.label, test_size = 0.1, shuffle = True, random_state = 42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
trainingset = np.loadtxt('convert_to_numpy.csv', delimiter=',', skiprows=1)
print(trainingset)
print(y_training)
sampleindex = np.random.randint(0,1000)
sample = trainingset[41996, :]
sample = sample.reshape(28, 28)

plt.imshow(sample, cmap='gray')
plt.show()
print(y_training.iloc[41996])

In [ ]:
print(np.unique(y_test))
naive = NB()
naive.fit(x_train, y_train)
predictedNB = naive.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedNB)),"\n")

In [ ]:
logReg = LR(solver='lbfgs', multi_class="multinomial").fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")

In [ ]:
# svmClass = SVC(kernel = 'linear')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
DTC = DecisionTreeClassifier(random_state=0)
DTC.fit(x_train, y_train)
predictedDTC = DTC.predict(x_test)
print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedDTC)), "\n")

In [ ]:
print(np.unique(y_test))
logReg = LR(solver='lbfgs', multi_class="ovr" ).fit(x_train, y_train)
predictedLR = logReg.predict(x_test)
print("The accuracy without applying PCA is: %.4f"%(metrics.accuracy_score(y_test, predictedLR)),"\n")
print()

# print()
# svmClass = SVC(kernel = 'rbf')
# svmClass.fit(x_train, y_train)
# predictedSVM = svmClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedSVM)), "\n")
# #print(confusion_matrix(bin_label_test, predictedSVM))

In [ ]:
print(np.unique(y_test))
KMClassList = []
for i in range(1, 11):
  KMClass = KMeans(n_clusters=i)
  KMClass.fit(x_train, y_train)
  predictedKMClass = KMClass.predict(x_test)
  print("The accuracy without applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedKMClass)), "\n")
  KMClassList.append(metrics.accuracy_score(y_test, predictedKMClass))

print(KMClassList)

# print(KMClassList)
# print(np.unique(y_test))
# MeanClass = MeanShift()
# MeanClass.fit(x_train, y_train)
# predictedMeanClass = MeanClass.predict(x_test)
# print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, predictedMeanClass)), "\n")

In [ ]:
print(np.unique(y_test))
AlgoClass = AgglomerativeClustering(n_clusters=10, affinity="euclidean", linkage="ward")
predictedAlgo = AlgoClass.fit(x_train, y_train)
AlgoClass.fit_predict(x_test)
print(AlgoClass.labels_)
print(y_test)

print("The accuracy after applying PCA is: %.4f" %(metrics.accuracy_score(y_test, AlgoClass.labels_)), "\n")

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)# np_utils.to_categorical(lb.fit_transform(y_train))
y_test = lb.fit_transform(y_test)# np_utils.to_categorical(lb.fit_transform(y_test))

print(x_train.shape)
display(y_train)
print(lb.classes_)

filename = 'labelsVox'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()
#display(y_train)
x_train = np.expand_dims(x_train, axis = 2)
x_test = np.expand_dims(x_test, axis = 2)
#x_train = np.squeeze(x_train)
#x_test = np.squeeze(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(x_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(10)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

In [ ]:
print(np.unique(y_test))
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(x_train, y_train, batch_size=50, epochs=25, validation_data=(x_test, y_test))

In [ ]:
print(np.unique(y_test))
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#----------------------------------------------#----------------------------------------------